<a href="https://colab.research.google.com/github/aj1365/WetMapFormer/blob/main/WetMapFormer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization,GlobalAveragePooling2D
from keras.layers import Dropout, Input
from keras.models import Model
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
# For running in python 2.x
from __future__ import print_function, unicode_literals
from __future__ import absolute_import, division
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Dropout, Dense, RepeatVector, Lambda, Reshape, Conv3D, Conv2D, Flatten, InputSpec
from keras.layers import BatchNormalization, Concatenate, Multiply, Add, Conv2DTranspose, GlobalAveragePooling2D, MaxPool2D
from keras.layers.advanced_activations import LeakyReLU, Softmax
from keras.models import Model
from tensorflow.keras import layers

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
def loadData(name):
    
    data_path = os.path.join(os.getcwd(),'E:/')
   
    if name == 'SA1':
        
        data = sio.loadmat(os.path.join(data_path, 'SA1.mat'))['SA1']
        labels = sio.loadmat(os.path.join(data_path, 'SA1_gt.mat'))['SA1_gt']
    if name == 'SA2':
        
        data = sio.loadmat(os.path.join(data_path, 'SA2.mat'))['SA2']
        labels = sio.loadmat(os.path.join(data_path, 'SA2_gt.mat'))['SA2_gt']
    if name == 'SA3':
        
        data = sio.loadmat(os.path.join(data_path, 'SA3.mat'))['SA3']
        labels = sio.loadmat(os.path.join(data_path, 'SA3_gt.mat'))['SA3_gt']
    
    return data, labels

In [ ]:
## GLOBAL VARIABLES
test_ratio = 0.3
windowSize = 8

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
dataset = 'SA1'
X1 , Y1 = loadData(dataset)
#X[X>100000]=-1
X1[np.isnan(X1)]=-1
X1[X1<-1000]=-1
X1.shape

In [ ]:
dataset = 'SA2'
X2 , Y2 = loadData(dataset)
#X[X>100000]=-1
X2[np.isnan(X2)]=-1
X2[X2<-1000]=-1
X2.shape

In [ ]:
dataset = 'SA3'
X3 , Y3 = loadData(dataset)
#X[X>100000]=-1
X3[np.isnan(X3)]=-1
X3[X3<-1000]=-1
X3.shape

In [ ]:
X1, Y1 = createImageCubes(X1, Y1, windowSize=windowSize)
X1.shape, Y1.shape

In [ ]:
X2, Y2 = createImageCubes(X2, Y2, windowSize=windowSize)
X2.shape, Y2.shape

In [ ]:
X3, Y3 = createImageCubes(X3, Y3, windowSize=windowSize)
X3.shape, Y3.shape

In [ ]:
X = np.concatenate((X1, X2, X3) , axis = 0)
Y = np.concatenate((Y1, Y2, Y3) , axis = 0)

X.shape,Y.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, Y, test_ratio)

np.min(ytrain), np.max(ytrain)

In [ ]:
Xtrain= Xtrain.reshape((Xtrain.shape[0],windowSize,windowSize,16))
image_size=8
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(Xtrain)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from keras_cv_attention_models.attention_layers import (
    activation_by_name,
    ChannelAffine,
    conv2d_no_bias,
    depthwise_conv2d_no_bias,
    drop_block,
    #MixupToken,
    mlp_block,
    output_block,
    add_pre_post_process,
)
from keras_cv_attention_models.download_and_load import reload_model_weights


In [ ]:
from keras_cv_attention_models.attention_layers import (
    ChannelAffine,
    CompatibleExtractPatches,
    conv2d_no_bias,
    drop_block,
    layer_norm,
    mlp_block,
    output_block,
    add_pre_post_process,
)
from keras_cv_attention_models.download_and_load import reload_model_weights


In [ ]:
BATCH_NORM_DECAY = 0.9
BATCH_NORM_EPSILON = 1e-5
LAYER_NORM_EPSILON = 1e-6

In [ ]:
class MultiHeadRelativePositionalKernelBias(tf.keras.layers.Layer):
    def __init__(self, input_height=-1, is_heads_first=False, **kwargs):
        super().__init__(**kwargs)
        self.input_height, self.is_heads_first = input_height, is_heads_first

    def build(self, input_shape):

        blocks, num_heads = (input_shape[2], input_shape[1]) if self.is_heads_first else (input_shape[1], input_shape[2])
        size = int(tf.math.sqrt(float(input_shape[-1])))
        height = self.input_height if self.input_height > 0 else int(tf.math.sqrt(float(blocks)))
        width = blocks // height
        pos_size = 2 * size - 1
        initializer = tf.initializers.truncated_normal(stddev=0.02)
        self.pos_bias = self.add_weight(name="positional_embedding", shape=(num_heads, pos_size * pos_size), initializer=initializer, trainable=True)

        idx_hh, idx_ww = tf.range(0, size), tf.range(0, size)
        coords = tf.reshape(tf.expand_dims(idx_hh, -1) * pos_size + idx_ww, [-1])
        bias_hh = tf.concat([idx_hh[: size // 2], tf.repeat(idx_hh[size // 2], height - size + 1), idx_hh[size // 2 + 1 :]], axis=-1)
        bias_ww = tf.concat([idx_ww[: size // 2], tf.repeat(idx_ww[size // 2], width - size + 1), idx_ww[size // 2 + 1 :]], axis=-1)
        bias_hw = tf.expand_dims(bias_hh, -1) * pos_size + bias_ww
        bias_coords = tf.expand_dims(bias_hw, -1) + coords
        bias_coords = tf.reshape(bias_coords, [-1, size**2])[::-1]  # torch.flip(bias_coords, [0])

        bias_coords_shape = [bias_coords.shape[0]] + [1] * (len(input_shape) - 4) + [bias_coords.shape[1]]
        self.bias_coords = tf.reshape(bias_coords, bias_coords_shape)  # [height * width, 1 * n, size * size]
        if not self.is_heads_first:
            self.transpose_perm = [1, 0] + list(range(2, len(input_shape) - 1))  # transpose [num_heads, height * width] -> [height * width, num_heads]

    def call(self, inputs):
        if self.is_heads_first:
            return inputs + tf.gather(self.pos_bias, self.bias_coords, axis=-1)
        else:
            return inputs + tf.transpose(tf.gather(self.pos_bias, self.bias_coords, axis=-1), self.transpose_perm)

    def get_config(self):
        base_config = super().get_config()
        base_config.update({"input_height": self.input_height, "is_heads_first": self.is_heads_first})
        return base_config

In [ ]:

def LWA(
    inputs, kernel_size=3, num_heads=2, key_dim=0, out_weight=True, qkv_bias=True, out_bias=True, attn_dropout=0, output_dropout=0, name=None
):
    
    _, hh, ww, cc = inputs.shape
    key_dim = key_dim if key_dim > 0 else cc // num_heads
    qk_scale = 1.0 / (float(key_dim) ** 0.5)
    out_shape = cc
    qkv_out = num_heads * key_dim

    should_pad_hh, should_pad_ww = max(0, kernel_size - hh), max(0, kernel_size - ww)
    if should_pad_hh or should_pad_ww:
        inputs = tf.pad(inputs, [[0, 0], [0, should_pad_hh], [0, should_pad_ww], [0, 0]])
        _, hh, ww, cc = inputs.shape

    qkv = keras.layers.Dense(qkv_out * 3, use_bias=qkv_bias, name=name and name + "qkv")(inputs)
    query, key_value = tf.split(qkv, [qkv_out, qkv_out * 2], axis=-1)  # Matching weights from PyTorch
    query = tf.expand_dims(tf.reshape(query, [-1, hh * ww, num_heads, key_dim]), -2)  # [batch, hh * ww, num_heads, 1, key_dim]

    key_value = CompatibleExtractPatches(sizes=kernel_size, strides=1, padding="VALID", compressed=False)(key_value)
    padded = (kernel_size - 1) // 2

    key_value = tf.concat([tf.repeat(key_value[:, :1], padded, axis=1), key_value, tf.repeat(key_value[:, -1:], padded, axis=1)], axis=1)
    key_value = tf.concat([tf.repeat(key_value[:, :, :1], padded, axis=2), key_value, tf.repeat(key_value[:, :, -1:], padded, axis=2)], axis=2)

    key_value = tf.reshape(key_value, [-1, kernel_size * kernel_size, key_value.shape[-1]])
    key, value = tf.split(key_value, 2, axis=-1)  # [batch * block_height * block_width, kernel_size * kernel_size, key_dim]
    key = tf.transpose(tf.reshape(key, [-1, key.shape[1], num_heads, key_dim]), [0, 2, 3, 1])  # [batch * hh*ww, num_heads, key_dim, kernel_size * kernel_size]
    key = tf.reshape(key, [-1, hh * ww, num_heads, key_dim, kernel_size * kernel_size])  # [batch, hh*ww, num_heads, key_dim, kernel_size * kernel_size]
    value = tf.transpose(tf.reshape(value, [-1, value.shape[1], num_heads, key_dim]), [0, 2, 1, 3])
    value = tf.reshape(value, [-1, hh * ww, num_heads, kernel_size * kernel_size, key_dim])  # [batch, hh*ww, num_heads, kernel_size * kernel_size, key_dim]

    attention_scores = keras.layers.Lambda(lambda xx: tf.matmul(xx[0], xx[1]))([query, key]) * qk_scale
    attention_scores = MultiHeadRelativePositionalKernelBias(input_height=hh, name=name and name + "pos")(attention_scores)
    attention_scores = keras.layers.Softmax(axis=-1, name=name and name + "attention_scores")(attention_scores)
    attention_scores = keras.layers.Dropout(attn_dropout, name=name and name + "attn_drop")(attention_scores) if attn_dropout > 0 else attention_scores


    attention_output = keras.layers.Lambda(lambda xx: tf.matmul(xx[0], xx[1]))([attention_scores, value])
    attention_output = tf.reshape(attention_output, [-1, hh, ww, num_heads * key_dim])


    if should_pad_hh or should_pad_ww:
        
        attention_output = attention_output[:, : hh - should_pad_hh, : ww - should_pad_ww, :]

    if out_weight:
        
        attention_output = keras.layers.Dense(out_shape, use_bias=out_bias, name=name and name + "output")(attention_output)
        
    attention_output = keras.layers.Dropout(output_dropout, name=name and name + "out_drop")(attention_output) if output_dropout > 0 else attention_output
    
    return attention_output



def block(inputs, out_channel, num_heads=0, attn_kernel_size=3, qkv_bias=True, mlp_ratio=4, mlp_drop_rate=0, attn_drop_rate=0, drop_rate=0, gamma=-1, activation="gelu"):
    is_conv = False if num_heads > 0 else True  # decide by if num_heads > 0
    
    input_channel = inputs.shape[-1]  # Same with out_channel
    
    pos_emb1 = depthwise_conv2d_no_bias(inputs, kernel_size=1, padding="SAME", use_bias=True)
    pos_emb2 = depthwise_conv2d_no_bias(inputs, kernel_size=3, padding="SAME", use_bias=True)
    pos_emb3 = depthwise_conv2d_no_bias(inputs, kernel_size=5, padding="SAME", use_bias=True)
    
    pos_out = keras.layers.Add()([inputs, pos_emb1, pos_emb2, pos_emb3])

    # print(f">>>> {is_conv = }, {num_heads = }")
    if is_conv:
        
        attn = keras.layers.BatchNormalization(momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(pos_out)
        attn = conv2d_no_bias(attn, out_channel, 1, use_bias=True)
        attn = depthwise_conv2d_no_bias(attn, kernel_size=5, padding="SAME", use_bias=True)
        attn = conv2d_no_bias(attn, out_channel, 1, use_bias=True)
        
    else:
        
        attn = layer_norm(inputs)
        attn = LWA(attn, attn_kernel_size, num_heads, attn_dropout=attn_drop_rate)
        attn = ChannelAffine(use_bias=False, weight_init_value=gamma)(attn) if gamma >= 0 else attn

    
    
    
    attn = drop_block(attn)
    attn_out = keras.layers.Add()([inputs, pos_out,  attn])

    if is_conv:
        
        mlp = keras.layers.BatchNormalization(momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(attn_out)
        
    else:
        
        mlp = keras.layers.LayerNormalization(epsilon=LAYER_NORM_EPSILON)(attn_out)
        
    mlp = mlp_block(mlp, int(out_channel * mlp_ratio), activation=activation)
    
    mlp = ChannelAffine(use_bias=False, weight_init_value=gamma)(mlp) if gamma >= 0 else mlp
    
    
    
    return keras.layers.Add()([inputs, attn_out, mlp])


def stem(inputs, stem_width, use_conv_stem=False, drop_rate=0, activation="gelu"):
    
    if use_conv_stem:
        
        nn = conv2d_no_bias(inputs, stem_width // 2, kernel_size=3, strides=2, padding="same", use_bias=True)
        nn = keras.layers.BatchNormalization(momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(nn)
        nn = activation_by_name(nn, activation, name=name)
        nn = conv2d_no_bias(nn, stem_width, kernel_size=3, strides=2, padding="same", use_bias=True,)
        nn = keras.layers.BatchNormalization(momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(nn)
        
    else:
        
        nn = conv2d_no_bias(inputs, stem_width, 4, strides=4, padding="valid", use_bias=True)
        nn = keras.layers.LayerNormalization(epsilon=LAYER_NORM_EPSILON)(nn)
        
    
    nn = keras.layers.Dropout(drop_rate) if drop_rate > 0 else nn
    
    return nn



def FExtractor(inputs):
    
    
    conv3d_shape = inputs.shape
    x = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3],1))(inputs)
    
    x = Conv3D(filters=16, kernel_size=(1, 1, 3), activation='relu', padding='same')(x)
    x = Conv3D(filters=32, kernel_size=(1, 1, 5), activation='relu', padding='same')(x)

    conv3d_shape = x.shape
    x = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(x)
    
    ##### #################################    UNet     
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual
   
    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(filters=32,kernel_size=(3, 3), padding="same")(x)
    #x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.SeparableConv2D(filters=32,kernel_size=(3, 3), padding="same")(x)
    #x = layers.BatchNormalization()(x)

    x1 = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
    residual = layers.Conv2D(filters=32, kernel_size=(3, 3),strides=2, padding="same")(
            previous_block_activation
        )
    x = layers.add([x1, residual])  # Add back residual
    previous_block_activation = x  # Set aside next residual


    x = layers.Activation("relu")(x1)
    x = layers.Conv2DTranspose(filters=32, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation("relu")(x)
    x = layers.Conv2DTranspose(filters=32, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.UpSampling2D(2)(x)

        # Project residual
    residual = layers.UpSampling2D(2)(previous_block_activation)
    residual = layers.Conv2D(filters=32, kernel_size=(3, 3), padding="same")(residual)
    x = layers.add([x, residual])  # Add back residual
 
    return x


def WetMapFormer(
    num_blocks=[3, 4],
    out_channels=[128, 256],
    head_dimension=64,
    use_conv_stem=False,
    block_types=["conv", "transform"],
    stem_width=-1,
    qkv_bias=True,
    mlp_ratio=4,
    layer_scale=-1,
    mix_token=False,
    token_label_top=False,
    input_shape=(8, 8, 16,1),
    num_classes=7,
    activation="gelu",
    mlp_drop_rate=0,
    attn_drop_rate=0,
    drop_connect_rate=0,
    dropout=0,
    classifier_activation="softmax",
    pretrained=None,
    model_name="WetMapFormer",
    kwargs=None,
):
    
    inputs = keras.layers.Input(input_shape)
    
    augmented = data_augmentation(inputs)
    
    x=FExtractor(augmented)

    
    """ stem """
    stem_width = stem_width if stem_width > 0 else out_channels[0]
    nn = stem(x, stem_width, use_conv_stem, drop_rate=mlp_drop_rate, activation=activation)  # It's using mlp_drop_rate for stem


    """ stage [1, 2, 3, 4] """
    total_blocks = sum(num_blocks)
    global_block_id = 0
    for stack_id, (num_block, out_channel, block_type) in enumerate(zip(num_blocks, out_channels, block_types)):
        stack_name = "stack{}_".format(stack_id + 1)
        is_conv_block = True if block_type[0].lower() == "c" else False
        num_heads = 0 if is_conv_block else out_channel // head_dimension
        if stack_id > 0:
            if use_conv_stem:
                nn = conv2d_no_bias(nn, out_channel, kernel_size=3, strides=2, padding="same", use_bias=True)
                nn = keras.layers.BatchNormalization(momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(nn)
            else:
                nn = conv2d_no_bias(nn, out_channel, kernel_size=2, strides=2, use_bias=True)
                nn = keras.layers.LayerNormalization(epsilon=LAYER_NORM_EPSILON)(nn)

        for block_id in range(num_block):
            block_drop_rate = drop_connect_rate * global_block_id / total_blocks
            nn = block(nn, out_channel, num_heads, qkv_bias, mlp_ratio, attn_drop_rate, block_drop_rate, layer_scale, activation)
            global_block_id += 1
    nn = keras.layers.BatchNormalization(momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(nn)

    """ output """
    if token_label_top and num_classes > 0:
        # Training with label token
        nn_cls = output_block(nn, num_classes=num_classes, classifier_activation=None)  # Don't use softmax here
        nn_aux = keras.layers.Dense(num_classes)(nn)

        if mix_token:
            
            nn_aux = mixup_token.do_mixup_token(nn_aux, bbox)
            nn_aux = keras.layers.Reshape((-1, nn_aux.shape[-1]), dtype="float32")(nn_aux)

            left, top, right, bottom = bbox
            lam = 1 - ((right - left) * (bottom - top) / (nn_aux.shape[1] * nn_aux.shape[2]))
            lam_repeat = tf.expand_dims(tf.repeat(lam, tf.shape(nn_cls)[0], axis=0), 1)
            nn_cls = keras.layers.Concatenate(axis=-1, dtype="float32")([nn_cls, lam_repeat])
            
        else:
            
            nn_aux = keras.layers.Reshape((-1, nn_aux.shape[-1]), dtype="float32")(nn_aux)
            
        out = [nn_cls, nn_aux]
        
    else:
        
        out = output_block(nn, num_classes=num_classes, classifier_activation=classifier_activation)

    model = keras.models.Model(inputs, out, name=model_name)
    
    return model



def WetMapFormer_small(input_shape=(8, 8, 16), num_classes=7, classifier_activation="softmax", token_label_top=False, **kwargs):
    
    num_blocks = [1, 1]
    head_dimension = 32
    
    return WetMapFormer(**locals(), model_name="WetMapFormer_small", **kwargs)

def WetMapFormer_medium(input_shape=(8, 8, 16), num_classes=7, classifier_activation="softmax", token_label_top=False, **kwargs):
    
    num_blocks = [3, 4]
    head_dimension = 256
    
    return WetMapFormer(**locals(), model_name="WetMapFormer_medium", **kwargs)


def WetMapFormer_large(input_shape=(8, 8, 16), num_classes=7, classifier_activation="softmax", token_label_top=False, **kwargs):
    
    num_blocks = [6, 8]
    head_dimension = 1024
    
    return WetMapFormer(**locals(), model_name="WetMapFormer_large", **kwargs)

In [ ]:

model = WetMapFormer_small(input_shape=(8, 8, 16), num_classes=7)
model.summary()


In [ ]:
import tensorflow_addons as tfa

In [ ]:
weight_decay = 0.0001
batch_size = 256
dropout_rate = 0.4
learning_rate = 0.001

In [ ]:
optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

checkpoint_filepath = "E:/WetMapFormer.h5"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

history = model.fit(
        x=Xtrain,
        y=ytrain,
        batch_size=batch_size,
        epochs=100,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

In [ ]:
Xtest = Xtest.reshape(-1, 8, 8, 16,1)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return average_acc


Y_pred = model.predict(Xtest)
y_pred = np.argmax(Y_pred, axis=1)
confusion = confusion_matrix(ytest, y_pred)
oa = accuracy_score(ytest, y_pred)
aa = AA_andEachClassAccuracy(confusion)
kappa = cohen_kappa_score(ytest, y_pred)


oa, aa, kappa